In [2]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import json
import re
import m3u8
import subprocess
import ffmpeg
import psycopg2
from datetime import datetime

ImportError: libpq.so.5: cannot open shared object file: No such file or directory

In [5]:
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"}

In [6]:
def parse_urls(page):
    host = "https://www.gamespot.com/news/"
    url = host + f"?page={page}"
    r = requests.get(url, headers=headers)
    soup = bs(r.content)
    return soup

In [7]:
def get_connection(): 
    return psycopg2.connect(dbname='news_classifier', user='postgres', 
                            password='Vadim14112003', host='localhost')

In [8]:
def save_article_link(data):
    try:
        cursor = connection.cursor()
        q = "INSERT INTO public.gamespot_article(url, article_title, publish, image_url) VALUES (%s, %s, %s, %s);"
        cursor.execute(q, (data['url'], data['title'], data['date'], data['iamge_url']))
        connection.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        connection.close()
    finally:
        cursor.close()

In [ ]:
def pars_urls():
    connection = get_connection()
    for i in range(1, 46):
        articles = parse_urls(i)

        articles = articles.findAll('div', class_='card-item')
        for image, body in articles:
            article = {
                'iamge_url': image.img['src'],
                'url': body.a['href'],
                'title': body.a.h4.text,
                'date': datetime.strptime(body.div.time['datetime'], '%A, %b %d, %Y %I:%M%p')
            }
            save_article_link(article)

    connection.close()

In [44]:
connection = get_connection()

In [45]:
try:
    cursor = connection.cursor()
    q = "SELECT * public.gamespot_article;"
    data = cursor.execute(q)
    data = cursor.fetchall()
except (Exception, psycopg2.DatabaseError) as error:
    connection.close()
finally:
    cursor.close()

In [46]:
connection.close()

In [49]:
data = [{
    'id': id_,
    'title': title,
    'url': url,
    'date_publish': date,
    'image_url': image,
} for id_, title, url, date, image in data]

In [55]:
print(list(filter(lambda x: x['date_publish'] != None, data))[0]['date_publish'])

2023-03-25


In [ ]:
def get_articles():
    